# Case Study EDA

### first steps

create dataframe, standardize columns and values


In [ ]:
import pandas as pd
import datetime as dt
import numpy as np

df = pd.read_csv('website_traffic_data.csv')
df.columns = [c.lower().strip().replace(' ','_').replace('-','_') for c in df.columns]
df['date'] = pd.to_datetime(df['date'], errors='coerce')
for col in ['bounce_rate','conversion_rate']:
    if df[col].dropna().between(1,100).mean() > 0.9: df[col] = df[col]/100

print(df.head())
print(df.columns)
print(df.dtypes)
print(df.isna().sum())

        date  sessions  bounce_rate  conversion_rate traffic_source  \
0 2025-04-01     246.0     0.564769         0.080461    Paid Search   
1 2025-04-02     474.0     0.657921         0.065349       Referral   
2 2025-04-03     524.0     0.453658         0.040685   Social Media   
3 2025-04-04     487.0     0.327508         0.038754    Paid Search   
4 2025-04-05     507.0     0.409198         0.021754       Referral   

      campaign  
0  Spring Sale  
1  Spring Sale  
2  Spring Sale  
3  Spring Sale  
4  Spring Sale  
Index(['date', 'sessions', 'bounce_rate', 'conversion_rate', 'traffic_source',
       'campaign'],
      dtype='object')
date               datetime64[ns]
sessions                  float64
bounce_rate               float64
conversion_rate           float64
traffic_source             object
campaign                   object
dtype: object
date               14
sessions           14
bounce_rate        14
conversion_rate    14
traffic_source     14
campaign           14


### feature engineering

In [2]:
df['conversions'] = (pd.to_numeric(df['sessions'], errors='coerce') *
                     pd.to_numeric(df['conversion_rate'], errors='coerce'))
df['week'] = df['date'].dt.to_period('W').dt.to_timestamp(how='start')
df['month'] = df['date'].dt.to_period('M').astype(str)
df['campaign_flag'] = (~df['campaign'].isna()) & (df['campaign'].str.lower().str.strip().ne("(none)")) & (df['campaign'].str.strip().ne(""))
df['is_weekend'] = df['date'].dt.weekday >= 5

### get basic metrics to keep track of
calculate rolling means and sums

In [4]:
daily = (df.dropna(subset=['date'])
        .groupby('date', as_index=False)
        .agg(sessions=('sessions', 'sum'),
        conversions=('conversions', 'sum'),
        bounce_rate=('bounce_rate', 'mean'),
        conversion_rate=('conversion_rate', 'mean'))
        .sort_values('date'))

daily['session_7d'] = daily['sessions'].rolling(7, min_periods=3).mean()
daily['bounce_rate_7d'] = daily['bounce_rate'].rolling(7, min_periods=3).mean()
daily['conversion_rate_7d'] = daily['conversion_rate'].rolling(7, min_periods=3).mean()

daily.head()

,date,sessions,conversions,bounce_rate,conversion_rate,session_7d,bounce_rate_7d,conversion_rate_7d
0,2025-04-01,246.0,19.793307,0.564769,0.080461,NaN,NaN,NaN
1,2025-04-02,474.0,30.975281,0.657921,0.065349,NaN,NaN,NaN
2,2025-04-03,524.0,21.319152,0.453658,0.040685,414.666667,0.558783,0.062165
3,2025-04-04,487.0,18.873319,0.327508,0.038754,432.750000,0.500964,0.056312
4,2025-04-05,507.0,11.029240,0.409198,0.021754,447.600000,0.482611,0.049401


### source scorecard

In [5]:
by_src = (df.groupby('traffic_source', dropna=False)
            .agg(sessions=("sessions", "sum"),
                 conversions=("conversions", "sum"),
                 bounce_rate=("bounce_rate", "mean"),
                 conversion_rate=("conversion_rate", "mean"))
            .reset_index())

by_src['traffic_share'] = by_src['sessions'] / by_src['sessions'].sum()
by_src.sort_values(['conversion_rate', 'sessions'], ascending=[False, False], inplace=True)
by_src

,traffic_source,sessions,conversions,bounce_rate,conversion_rate,traffic_share
3,Referral,17577.0,949.280364,0.501025,0.054491,0.189244
0,Direct,21072.0,1048.840181,0.517865,0.049911,0.226873
1,Organic Search,17732.0,876.335159,0.483879,0.048688,0.190913
4,Social Media,18545.0,898.077123,0.489995,0.047433,0.199666
2,Paid Search,17954.0,801.146310,0.504093,0.045257,0.193303
5,NaN,0.0,0.000000,NaN,NaN,0.000000


### campaign scorecard

In [6]:
by_campaign = (df.groupby('campaign', dropna=False)
            .agg(sessions=("sessions", "sum"),
                 conversions=("conversions", "sum"),
                 bounce_rate=("bounce_rate", "mean"),
                 conversion_rate=("conversion_rate", "mean"))
            .reset_index()
            .sort_values(['conversions', 'conversion_rate'], ascending=[False, False]))

by_campaign

,campaign,sessions,conversions,bounce_rate,conversion_rate
1,Spring Sale,31174.0,1633.344411,0.495306,0.052536
2,Summer Launch,30725.0,1528.175548,0.517112,0.049834
0,Back to School,30981.0,1412.159177,0.488176,0.045216
3,NaN,0.0,0.000000,NaN,NaN


### stability and day of week patterns

In [ ]:
cvr_daily_src = (df.dropna(subset='date')
                 .groupby(['traffic_source', 'date'], as_index=False)
                 .agg(conversion_rate=('conversion_rate','mean')))

stability_src = (cvr_daily_src.groupby('traffic_source', as_index=False)
                 .agg(cvr_mean=('conversion_rate', 'mean'),
                      cvr_std=('conversion_rate', 'std'),
                      days=('date', 'nunique')))

stability_src['stability_score'] = stability_src['crv_mean'] / stability_src['cvr_std']
stability_src.sort_values('stability_score', ascending=False, inplace=True)
print('Source Stability')
stability_src


df['dow'] = df['date'].dt.weekday
dow_profile = (df.dropna(subset=["dow"])
                 .groupby("dow", as_index=False)
                 .agg(sessions=("sessions", "sum"),
                      conversions=("conversions", "sum"),
                      bounce_rate=("bounce_rate", "mean"),
                      conversion_rate=("conversion_rate", "mean")))
print('Day of Week Patterns')
dow_profile

corr = df[["bounce_rate", "conversion_rate"]].corr(method="spearman").iloc[0,1]
print(f"Spearman(bounce_rate, conversion_rate) = {corr:.3f}")

### campaign lift   

In [ ]:
from datetime import timedelta

def campaign_lift(df, campagin_name, pre_days=14):
    sub = df[df['campaign'] == campagin_name].dropna(subset=['date'])
    if sub.empty:
        return None
    start_date = sub['date'].min()
    pre_mask = (df['date'] >= (start_date - timedelta(days=pre_days))) & (df['date'] < start_date)
    during_mask = (df['campaign'] == campagin_name)

    pre = df.loc[pre_mask, 'conversion_rate'].mean()
    during = df.loc[during_mask, 'conversion_rate'].mean()

    lift = np.nan
    if pd.notna(pre) and pre > 0:
        lift = (during - pre) / pre

    return {
        'campaign': campagin_name,
        'start_date': start_date,
        'pre_cvr': pre,
        'during_cvr': during,
        'lift_vs_pre': lift
    }

lift_rows = [campaign_lift(df, c) for c in df['campaign'].dropna().unique()]
lift_df = pd.DataFrame([row for row in lift_rows if row is not None]).sort_values('lift_vs_pre', ascending=False, inplace=True)
print('Campaign Lift Analysis')
lift_df

### opportunity sizing

In [ ]:
def relocation_delta(by_src, shift_frac=0.10, top_k=1, bottom_k=1):
    src = by_src.copy()
    src.sort_values(['conversion_rate', 'sessions'], ascending=[False, False], inplace=True)
    best= src.head(top_k)
    worst = src.tail(bottom_k)

    delta_sessions = worst['sessions'].sum() * shift_frac
    cvr_best = (best["conversion_rate"] * best["sessions"]).sum() / max(best["sessions"].sum(), 1)
    cvr_worst = (worst["conversion_rate"] * worst["sessions"]).sum() / max(worst["sessions"].sum(), 1)
    delta_conversions = delta_sessions * (cvr_best - cvr_worst)

    return {
        'shift_frac': shift_frac,
        'from_sources': worst['traffic_source'].tolist(),
        'to_sources': best['traffic_source'].tolist(),
        'delta_sessions': delta_sessions,
        'cvr_best': cvr_best,
        'cvr_worst': cvr_worst,
        'delta_conversions': delta_conversions
    }

senario_10 = relocation_delta(by_src, shift_frac=0.10, top_k=1, bottom_k=1)
senario_20 = relocation_delta(by_src, shift_frac=0.20, top_k=1, bottom_k=1)
print('Opportunity Sizing via Traffic Source Relocation')
pd.DataFrame([senario_10, senario_20])

### generate plots

In [ ]:
import matplotlib.pyplot as plt

#daily sessions plot
fig = plt.figure(figsize=(10,5))
plt.plot(daily["date"], daily["sessions"], label="Sessions (daily)")
plt.plot(daily["date"], daily["sessions_7d"], label="Sessions (7d avg)")
plt.title("Daily Sessions")
plt.xlabel("Date"); plt.ylabel("Sessions")
plt.legend(); plt.tight_layout()
plt.savefig("chart_daily_sessions.png", dpi=200)

In [ ]:
#daily bounce and cvr plot
fig = plt.figure(figsize=(10,5))
plt.plot(daily["date"], daily["bounce_rate_7d"], label="Bounce Rate (7d)")
plt.plot(daily["date"], daily["conversion_rate_7d"], label="Conversion Rate (7d)")
plt.title("Quality Trends (7-day smoothed)")
plt.xlabel("Date"); plt.ylabel("Rate")
plt.legend(); plt.tight_layout()
plt.savefig("chart_quality_trends.png", dpi=200)

In [ ]:
#source scorecard
src_plot = by_src.sort_values("conversion_rate", ascending=False)
fig = plt.figure(figsize=(10,5))
plt.bar(src_plot["traffic_source"].astype(str), src_plot["conversion_rate"])
plt.title("Conversion Rate by Source")
plt.xlabel("Traffic Source"); plt.ylabel("Conversion Rate")
plt.tight_layout()
plt.savefig("chart_cvr_by_source.png", dpi=200)

In [ ]:
#scatter bounce vs cvr
fig = plt.figure(figsize=(7,6))
sizes = (by_src["sessions"] / by_src["sessions"].max()) * 1000  # scale bubble size
plt.scatter(by_src["bounce_rate"], by_src["conversion_rate"], s=sizes)
for _, r in by_src.iterrows():
    plt.text(r["bounce_rate"], r["conversion_rate"], str(r["traffic_source"]))
plt.xlabel("Bounce Rate"); plt.ylabel("Conversion Rate")
plt.title("Bounce vs Conversion by Source (bubble size = sessions)")
plt.tight_layout()
plt.savefig("chart_bounce_vs_cvr.png", dpi=200)